In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from pandas import read_parquet
import random
song_set = set()
test_df = pd.read_csv("/content/drive/MyDrive/datagame/label_test_source.csv")  # 也可以直接讀parquet，這邊只是先另存成csv
test_df.sort_values(by=['session_id', "listening_order"], inplace=True)
test_df.reset_index(inplace=True, drop=True)
song = read_parquet("/content/drive/MyDrive/datagame/meta_song.parquet")

In [ ]:
# 存入所有的song_id (不重複)
for i in range(len(test_df)):
    song_set.add(test_df.song_id[i])
for i in range(len(song)):
    song_set.add(song.song_id[i])
song_set = list(song_set)

In [ ]:
output = {"session_id":[], "top1":[], "top2":[], "top3":[], "top4":[], "top5":[]}
global_used = set()
for i in range(len(test_df) // 20):
    tmp = {}
    # count song_id in session
    for j in test_df.loc[i * 20:(i + 1) * 20, "song_id"]:
        if j in tmp:
            tmp[j] += 1
        else:
            tmp[j] = 1
    list_of_tuples = list(tmp.items())
    list_of_tuples.sort(key=lambda x: x[1], reverse=True)

    top_5_keys = []
    used = []
    stuck_idx = 0
    for idx, x in enumerate(list_of_tuples):
        if x[1] == 1:  # if only play 1 time
            if stuck_idx == 0:
                stuck_idx = idx
            choose = random.randint(stuck_idx, len(list_of_tuples) - 1)  # 從前面只聽一次的歌曲裡面隨機選一個沒選過的
            while choose in used:  # if already used, random another index
                choose = random.randint(stuck_idx, len(list_of_tuples) - 1)
            top_5_keys.append(list_of_tuples[choose][0])
            global_used.add(list_of_tuples[choose][0])
            used.append(choose)
        else:
            top_5_keys.append(x[0])
            used.append(idx)
            global_used.add(x[0])
        if len(top_5_keys) == 5 or len(top_5_keys) == len(list_of_tuples):
            break

    for j in range(5 - len(top_5_keys)):  # 如果還不滿五首，就先補0
        top_5_keys.append(0)

    output["session_id"].append(test_df.loc[i * 20, "session_id"])
    for j in range(1, 6):
        output["top" + str(j)].append(top_5_keys[j - 1])

In [ ]:
# 對於剛剛補0的部分，替換成隨機一首沒出現過的歌曲
for i in range(1, 6):
    for j in range(len(output["top" + str(i)])):
        if output["top" + str(i)][j] == 0:
            choose_song = random.choice(song_set)
            while choose_song in global_used:  # 如果已經使用過，就再隨機選一首
                choose_song = random.choice(song_set)
            output["top" + str(i)][j] = choose_song
            global_used.add(choose_song)

In [ ]:
pd.DataFrame(output).to_csv("/content/drive/MyDrive/datagame/sort_source.csv", index=False)